In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import csv
from tqdm import tqdm

In [2]:
# import data
df = pd.read_csv('00_df.csv', index_col=0)
pre_df = pd.read_csv('01_pre_df.csv', index_col=0)
interpoltaed_mv_df = pd.read_csv('02_interpoltaed_mv_df.csv', index_col=0)

In [3]:
df.shape   # (783926, 41)

(783926, 41)

In [4]:
pre_df.shape   # (783926, 37)

(783926, 37)

In [5]:
interpoltaed_mv_df.shape

(783926, 40)

In [6]:
# import list data

data_length_per_person = []
data_length_per_person_sum = []

with open('data_length_per_person.csv', 'r', encoding='utf-8') as f:
    rdr = csv.reader(f)
    for para in rdr:
        for i in para:
            data_length_per_person.append(int(i))


with open('data_length_per_person_sum.csv', 'r', encoding='utf-8') as f:
    rdr = csv.reader(f)
    for para in rdr:
        for i in para:
            data_length_per_person_sum.append(int(i))

In [7]:
len(data_length_per_person)   # 20169

20169

In [8]:
len(data_length_per_person_sum)   # 20169

20169

## interpoltaed_mv_df 기준 
논적으로 필요없는 데이터와 선형회귀로 결측치가 대치된 데이터만 사용해서 테스트

In [9]:
interpoltaed_mv_df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,FiO2,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,101.6,91.5,36.80,126.8,87.30,NaN,24.8,20.7,46.5,0.30,...,10.2,NaN,327.5,83.14,0,NaN,NaN,-0.03,1,0
1,97.0,95.0,36.80,98.0,75.33,NaN,19.0,20.7,46.5,0.30,...,10.2,NaN,327.5,83.14,0,NaN,NaN,-0.03,2,0
2,89.0,99.0,36.80,122.0,86.00,NaN,22.0,20.7,46.5,0.30,...,10.2,NaN,327.5,83.14,0,NaN,NaN,-0.03,3,0
3,90.0,95.0,36.80,126.8,87.30,NaN,30.0,24.0,46.5,0.30,...,10.2,NaN,327.5,83.14,0,NaN,NaN,-0.03,4,0
4,103.0,88.5,36.80,122.0,91.33,NaN,24.5,20.7,46.5,0.28,...,10.2,NaN,327.5,83.14,0,NaN,NaN,-0.03,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783921,88.0,98.0,38.10,135.0,81.00,64.0,16.0,0.0,25.7,0.50,...,14.0,NaN,325.5,62.29,1,NaN,NaN,-0.03,31,1
783922,96.0,98.0,38.72,174.0,97.00,72.0,16.0,2.0,25.7,0.50,...,14.0,NaN,325.5,62.29,1,NaN,NaN,-0.03,32,1
783923,140.0,97.0,38.10,133.0,81.50,62.5,16.0,0.0,25.7,0.50,...,14.0,NaN,325.5,62.29,1,NaN,NaN,-0.03,33,1
783924,120.0,96.0,38.10,154.0,118.00,105.0,16.0,0.0,25.7,0.50,...,14.0,NaN,325.5,62.29,1,NaN,NaN,-0.03,34,1


In [10]:
cols = interpoltaed_mv_df.columns
cols

Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'BaseExcess',
       'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos',
       'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct', 'Glucose',
       'Lactate', 'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total',
       'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC', 'Fibrinogen', 'Platelets',
       'Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS',
       'SepsisLabel'],
      dtype='object')

In [17]:
interpoltaed_mv_df.drop(['Unit1', 'Unit2', 'HospAdmTime', 'ICULOS'], axis=1, inplace=True)

In [24]:
interpoltaed_mv_df.shape

(783926, 36)

In [32]:
cols = interpoltaed_mv_df.columns
len(cols)

36

## 환자별 20 row data를 추출
20 row가 되지 않는다면 drop / 20 이상의 row를 가지고 있다면 뒤쪽 데이터부터 20을 추출

In [26]:
len(data_length_per_person_sum)

20169

In [27]:
data_length_per_person_sum.insert(0,0)

In [29]:
len(data_length_per_person_sum)

20170

In [40]:
s_i = 0
e_i = 1

data_per_patient_for_20 = pd.DataFrame(columns = cols)
for i in tqdm(range(len(data_length_per_person_sum)-1)):
    s = data_length_per_person_sum[s_i]
    e = data_length_per_person_sum[e_i]
    if e - s >= 20:
        temp = interpoltaed_mv_df[e-20:e]
        data_per_patient_for_20 = pd.concat([data_per_patient_for_20, temp])
    
    s_i += 1
    e_i += 1

100%|██████████| 20169/20169 [08:32<00:00, 39.37it/s]


In [41]:
data_per_patient_for_20

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,FiO2,...,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,SepsisLabel
34,102.0,91.5,36.80,103.0,75.67,NaN,24.0,20.7,46.5,0.3,...,NaN,36.7,12.3,NaN,10.2,NaN,327.5,83.14,0,0
35,99.0,89.0,37.11,113.0,96.33,NaN,26.0,20.7,46.5,0.3,...,NaN,36.7,12.3,NaN,10.2,NaN,327.5,83.14,0,0
36,97.0,89.0,36.80,110.0,72.00,NaN,29.0,20.7,46.5,0.3,...,NaN,36.7,12.3,NaN,10.2,NaN,327.5,83.14,0,0
37,110.0,89.0,36.80,108.0,90.67,NaN,24.0,20.7,46.5,0.3,...,NaN,36.7,12.3,NaN,10.2,NaN,327.5,83.14,0,0
38,108.0,89.0,36.80,139.0,102.33,NaN,26.0,20.0,46.5,0.3,...,NaN,36.7,12.3,NaN,10.2,NaN,327.5,83.14,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783921,88.0,98.0,38.10,135.0,81.00,64.0,16.0,0.0,25.7,0.5,...,NaN,31.2,11.6,29.8,14.0,NaN,325.5,62.29,1,1
783922,96.0,98.0,38.72,174.0,97.00,72.0,16.0,2.0,25.7,0.5,...,NaN,27.8,11.6,29.8,14.0,NaN,325.5,62.29,1,1
783923,140.0,97.0,38.10,133.0,81.50,62.5,16.0,0.0,25.7,0.5,...,NaN,31.2,11.6,29.8,14.0,NaN,325.5,62.29,1,1
783924,120.0,96.0,38.10,154.0,118.00,105.0,16.0,0.0,25.7,0.5,...,NaN,31.2,11.6,29.8,14.0,NaN,325.5,62.29,1,1


In [51]:
people = int(data_per_patient_for_20.shape[0]/20)
people

17585.0

In [60]:
diff = (len(data_length_per_person_sum)-1) - data_per_patient_for_20.shape[0]/20
loss = round(diff/(len(data_length_per_person_sum)-1)*100, 1)

print(f'20행 미만의 데이터 수 : {diff}, 손실율: {loss})')

20행 미만의 데이터 수 : 2584.0, 손실율: 12.8)


### sepsis vs non_sepsis 추출

In [62]:
# sepsis와 non sepsis 구분

s = 0
e = 20

sepsis_df = pd.DataFrame(columns=cols)
non_sepsis_df = pd.DataFrame(columns=cols)

for _ in tqdm(range(people)):
    if data_per_patient_for_20[s:e]['SepsisLabel'].sum() >= 1:
        data = data_per_patient_for_20[s:e]
        sepsis_df = pd.concat([sepsis_df, data])
    else: 
        data = data_per_patient_for_20[s:e]
        non_sepsis_df = pd.concat([non_sepsis_df, data])
        
    s += 20
    e += 20

100%|██████████| 17585/17585 [07:33<00:00, 38.79it/s]


In [107]:
sepsis_df.shape

(24700, 36)

In [64]:
non_sepsis_df.shape

(327000, 36)

## 3시간의 데이터를 보고 6시간 후의 패혈증 발병 정도를 예측
### label 우선 추출
1환자당 12개의 데이터 추출가능

In [85]:
e = 20

s_label = []
for i in tqdm(range(int(sepsis_df.shape[0]/20))):
    temp = sepsis_df["SepsisLabel"][e-12:e]
    s_label.append(list(temp))
    
    e += 20

100%|██████████| 1235/1235 [00:00<00:00, 14736.20it/s]


In [92]:
e = 20

ns_label = []
for i in tqdm(range(int(non_sepsis_df.shape[0]/20))):
    temp = non_sepsis_df["SepsisLabel"][e-12:e]
    ns_label.append(list(temp))
    
    e += 20

100%|██████████| 16350/16350 [00:00<00:00, 16659.78it/s]


### 훈련, 검증, 테스트 세트 구성
sepsis와 non_sepsis 데이터의 비율을 고려하지 않고 모든 데이터를 활용

In [189]:
sepsis_df_no_label = sepsis_df.drop(['SepsisLabel'], axis=1)
sepsis_df_no_label.shape

(24700, 35)

In [190]:
non_sepsis_df_no_label = non_sepsis_df.drop(['SepsisLabel'], axis=1)
non_sepsis_df_no_label.shape

(327000, 35)

### 선형회귀법으로 결측치를 대치하고 남은 결측치는 현 상태의 평균대치법으로 적용

In [191]:
cols = sepsis_df_no_label.columns

In [192]:
# # 전체 평균

# total_mean = []

# for col in cols:
#     total_mean.append(round(sepsis_df_no_label[col].mean(), 1))

# len(total_mean)

35

In [225]:
# 환자 별 평균
person_mean = []

s_i = 0
e_i = 20

for _ in tqdm(range(int(sepsis_df_no_label.shape[0]/20))):
    temp = []
    for col in cols:
        temp.append(sepsis_df_no_label[col][s_i:e_i].mean())
    person_mean.append(temp)
    
    s_i += 20
    e_i += 20
    
len(person_mean)

100%|██████████| 1235/1235 [00:06<00:00, 182.32it/s]


1235

### 환자별 평균 값으로 column을 채워도 결측인 값은 0으로 대치 / 이후 전체 평균으로 대치하는 방법도 시도

In [228]:
# 환자별 평균을 정리한 리스트에 결측값을 0으로 변환

for i in tqdm(range(len(person_mean))):
    for j in range(len(person_mean[i])):
        if str(person_mean[i][j]) == "nan":
            person_mean[i][j] = 0
        else:
            person_mean[i][j] = round(person_mean[i][j], 1)

100%|██████████| 1235/1235 [00:00<00:00, 9310.62it/s]


In [255]:
# DP 생성 후 결측치 없는 데이터 세트 구성

sepsis_df_no_label_mv = pd.DataFrame(columns = cols)

s_i = 0
e_i = 20

for i in tqdm(range(int(sepsis_df_no_label.shape[0]/20))):
    temp = pd.DataFrame()
    for j, col in enumerate(cols):
        temp[col] = sepsis_df_no_label[col][s_i:e_i].fillna(person_mean[i][j])
        
    sepsis_df_no_label_mv = pd.concat([sepsis_df_no_label_mv, temp])
    s_i += 20
    e_i += 20

100%|██████████| 1235/1235 [00:41<00:00, 29.56it/s]


### 1환자 데이터(20개)를 3개씩 묶어서 x_train 세트 구성

In [269]:
def ratio(li):
    l = len(li)
    train = l*0.8
    val = train + int(l*0.1)
    return int(train), int(val)

In [270]:
tr, va = ratio(s_label)
print(tr, va)

988 1111


In [279]:
# sepsis 데이터 > x_train, x_val, x_test

s = 0
x_train = []
x_val = []
x_test = []

for _ in tqdm(range(int(sepsis_df_no_label_mv.shape[0]/20))):
    if _ < 988:
        for i in range(12):     # 20개의 데이터 세트에서 12개의 데이터를 label에 달아 사용 가능
            temp = sepsis_df_no_label_mv[s+i : s+i+3].values.tolist()
            x_train.append(temp)
    elif _ < 1111:
        for i in range(12):     # 20개의 데이터 세트에서 12개의 데이터를 label에 달아 사용 가능
            temp = sepsis_df_no_label_mv[s+i : s+i+3].values.tolist()
            x_val.append(temp)
    else: 
        for i in range(12):     # 20개의 데이터 세트에서 12개의 데이터를 label에 달아 사용 가능
            temp = sepsis_df_no_label_mv[s+i : s+i+3].values.tolist()
            x_test.append(temp)
    s += 20

100%|██████████| 1235/1235 [00:11<00:00, 109.08it/s]


In [280]:
print(len(x_train)/12, len(x_val)/12, len(x_test)/12, len(x_train)//12 + len(x_val)//12 + len(x_test)//12)

988.0 123.0 124.0 1235


### non_sepsis_df 에도 동일하게 수행

In [281]:
# 환자 별 평균
person_mean = []

s_i = 0
e_i = 20

for _ in tqdm(range(int(non_sepsis_df_no_label.shape[0]/20))):
    temp = []
    for col in cols:
        temp.append(non_sepsis_df_no_label[col][s_i:e_i].mean())
    person_mean.append(temp)
    
    s_i += 20
    e_i += 20
    
len(person_mean)

100%|██████████| 16350/16350 [01:28<00:00, 184.59it/s]


16350

In [282]:
# 환자별 평균을 정리한 리스트에 결측값을 0으로 변환

for i in tqdm(range(len(person_mean))):
    for j in range(len(person_mean[i])):
        if str(person_mean[i][j]) == "nan":
            person_mean[i][j] = 0
        else:
            person_mean[i][j] = round(person_mean[i][j], 1)

100%|██████████| 16350/16350 [00:01<00:00, 11334.24it/s]


In [284]:
# DP 생성 후 결측치 없는 데이터 세트 구성

non_sepsis_df_no_label_mv = pd.DataFrame(columns = cols)

s_i = 0
e_i = 20

for i in tqdm(range(int(non_sepsis_df_no_label.shape[0]/20))):
    temp = pd.DataFrame()
    for j, col in enumerate(cols):
        temp[col] = non_sepsis_df_no_label[col][s_i:e_i].fillna(person_mean[i][j])
        
    non_sepsis_df_no_label_mv = pd.concat([non_sepsis_df_no_label_mv, temp])
    s_i += 20
    e_i += 20

100%|██████████| 16350/16350 [13:52<00:00, 19.64it/s]


In [293]:
# non_sepsis 데이터 > y_train, y_val, y_test

s = 0

for _ in tqdm(range(int(non_sepsis_df_no_label_mv.shape[0]/20))):
    if _ < 988:
        for i in range(12):     # 20개의 데이터 세트에서 12개의 데이터를 label에 달아 사용 가능
            temp = non_sepsis_df_no_label_mv[s+i : s+i+3].values.tolist()
            x_train.append(temp)
    elif _ < 1111:
        for i in range(12):     # 20개의 데이터 세트에서 12개의 데이터를 label에 달아 사용 가능
            temp = non_sepsis_df_no_label_mv[s+i : s+i+3].values.tolist()
            x_val.append(temp)
    elif _ < 1235:
        for i in range(12):     # 20개의 데이터 세트에서 12개의 데이터를 label에 달아 사용 가능
            temp = non_sepsis_df_no_label_mv[s+i : s+i+3].values.tolist()
            x_test.append(temp)
    else:
        break
    s += 20

  8%|▊         | 1235/16350 [00:12<02:33, 98.38it/s] 


In [294]:
print(len(x_train)/12, len(x_val)/12, len(x_test)/12, len(x_train)//12 + len(x_val)//12 + len(x_test)//12)

1976.0 246.0 248.0 2470


In [308]:
print(len(x_train)/12, len(x_val)/12, len(x_test)/12, len(x_train)//12 + len(x_val)//12 + len(x_test)//12)

1976.0 246.0 248.0 2470


In [303]:
# sepsis label > train, val, test set 구성

y_train = []
y_val = []
y_test = []   # 988 123 124 1235

for _ in range(len(s_label)):
    if _ < 988:
        y_train += s_label[_]
    elif _ < 1111:
        y_val += s_label[_]
    elif _ < 1235:
        y_test += s_label[_]
    else:
        break

In [309]:
print(len(y_train)/12, len(y_val)/12, len(y_test)/12, len(y_train)//12 + len(y_val)//12 + len(y_test)//12)

988.0 123.0 124.0 1235


In [312]:
y_train += [0] * len(y_train)
y_val += [0] * len(y_val)
y_test += [0] * len(y_test)

In [316]:
print(len(y_train)/12, len(y_val)/12, len(y_test)/12, len(y_train)//12 + len(y_val)//12 + len(y_test)//12)

1976.0 246.0 248.0 2470


## 파이선 list로 되어있는 데이터 세트를 np.array로 변환

In [330]:
X_train = np.array(x_train)
X_train

array([[[133.  ,  94.  ,  37.6 , ..., 263.5 ,  27.92,   1.  ],
        [137.  ,  94.  ,  37.6 , ..., 263.5 ,  27.92,   1.  ],
        [138.  ,  96.  ,  39.33, ..., 263.5 ,  27.92,   1.  ]],

       [[137.  ,  94.  ,  37.6 , ..., 263.5 ,  27.92,   1.  ],
        [138.  ,  96.  ,  39.33, ..., 263.5 ,  27.92,   1.  ],
        [140.  ,  96.  ,  37.6 , ..., 263.5 ,  27.92,   1.  ]],

       [[138.  ,  96.  ,  39.33, ..., 263.5 ,  27.92,   1.  ],
        [140.  ,  96.  ,  37.6 , ..., 263.5 ,  27.92,   1.  ],
        [135.  ,  97.  ,  38.67, ..., 263.5 ,  27.92,   1.  ]],

       ...,

       [[ 93.  , 100.  ,  37.  , ..., 194.  ,  65.99,   1.  ],
        [ 93.  , 100.  ,  37.  , ..., 194.  ,  65.99,   1.  ],
        [ 94.  , 100.  ,  37.67, ..., 194.  ,  65.99,   1.  ]],

       [[ 93.  , 100.  ,  37.  , ..., 194.  ,  65.99,   1.  ],
        [ 94.  , 100.  ,  37.67, ..., 194.  ,  65.99,   1.  ],
        [ 91.  , 100.  ,  37.  , ..., 194.  ,  65.99,   1.  ]],

       [[ 94.  , 100.  ,  37.67,

In [331]:
X_val = np.array(x_val)
X_test = np.array(x_test)

Y_train = np.array(y_train)
Y_val = np.array(y_val)
Y_test = np.array(y_test)

# 데이터 세트 구성 완료
## 모델은 이전과 동일하게 구성

In [321]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [345]:
hidden_units = 128

model = Sequential()
model.add(LSTM(hidden_units, activation='relu', input_shape=(3, 35)))
model.add(Dense(64))
model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 128)               83968     
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 92,289
Trainable params: 92,289
Non-trainable params: 0
_________________________________________________________________


In [346]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [347]:
hist = model.fit(X_train, Y_train, epochs=5, batch_size=128, validation_data=(X_val, Y_val))

Epoch 1/5
186/186 [==============================] - 4s 11ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 2/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 3/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 4/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 5/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028


학습 안됨. 

1. 초기 데이터 세트(df)에서 선형회귀 방법으로 결측치를 대치하고 변화되지 않는 결측값은 0으로 처리.
2. sepsis와 non_sepsis군을 나누어 전처리를 진행하였고, 3시간의 데이터를 보고 6시간 후 sepsis 확률을 예측할 수 있는 모습으로 전처리
3. sepsis 데이터량이 non_sepsis에 비해 많이 적어 sepsis 양에 맞게 non_sepsis 데이터를 제한하여 train, val, test를 구성 
4. 기본적인 LSTM의 모델을 구성하여 학습을 진행
5. 학습 안됨

In [358]:
result = model.predict(X_test)
print('## result ##')

o = 0
x = 0

for y, r in zip(Y_test, result):
    if y == r:
        o += 1
    else:
        x += 1
    
print(o, x, o+x)

## result ##
1199 1777 2976


In [363]:
print(o/(o+x), x/(o+x))

0.40288978494623656 0.5971102150537635


### 의문
>혹시 x_train값이 3차원으로 구성되어 있는데 y_train의 값이 2차원이라 학습이 안되는가? 확인

In [367]:
test = Y_train.reshape(-1, 1)
test

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [368]:
t_val = Y_val.reshape(-1, 1)

In [369]:
hist = model.fit(X_train, test, epochs=5, batch_size=128, validation_data=(X_val, t_val))

Epoch 1/5
186/186 [==============================] - 3s 12ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 2/5
186/186 [==============================] - 2s 10ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 3/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 4/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 5/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028


loss가 확인되지 않음.

### 의문
>혹시 shuffle이 안되어서 학습이 안될 수 도 있기 때문에 shuffle 진행 후 다시 시도

In [378]:
## Shuffle 기능 확인

import numpy as np

# x = np.arange(10).reshape(-1, 1)
# print ("x: ", x)
y = np.arange(10).reshape(-1, 1)
# print ("y: ", y)

x = np.array([[0,1], [1,2], [2,3], [3,4], [4,5], [5,6], [6,7], [7,8], [8,9], [9,10]])

s = np.arange(x.shape[0])
print('s', s)
np.random.shuffle(s)

x = x[s]
print ("x: ", x)
y = y[s]
print ("y: ", y)

s [0 1 2 3 4 5 6 7 8 9]
x:  [[ 4  5]
 [ 2  3]
 [ 9 10]
 [ 6  7]
 [ 0  1]
 [ 8  9]
 [ 1  2]
 [ 5  6]
 [ 3  4]
 [ 7  8]]
y:  [[4]
 [2]
 [9]
 [6]
 [0]
 [8]
 [1]
 [5]
 [3]
 [7]]


In [381]:
# Train, Test data 적용
i = np.arange(X_train.shape[0])
np.random.shuffle(i)

train = X_train[i]
test = Y_train[i]

In [382]:
# Val data 적용

i = np.arange(X_val.shape[0])
np.random.shuffle(i)

sx_val = X_val[i]
sy_val = Y_val[i]

In [383]:
hist = model.fit(train, test, epochs=5, batch_size=128, validation_data=(sx_val, sy_val))

Epoch 1/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 2/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 3/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 4/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 5/5
186/186 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028


변화 없음

### 의문
> 혹시 node가 너무 적어서 학습이 안되는지 확인

In [386]:
hidden_units = 512

model = Sequential()
model.add(LSTM(hidden_units, activation='relu', input_shape=(3, 35)))
model.add(Dense(64))
model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 512)               1122304   
                                                                 
 dense_12 (Dense)            (None, 64)                32832     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,155,201
Trainable params: 1,155,201
Non-trainable params: 0
_________________________________________________________________


In [387]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [389]:
hist = model.fit(train, test, epochs=5, batch_size=128, validation_data=(sx_val, sy_val))

Epoch 1/5
186/186 [==============================] - 10s 52ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 2/5
186/186 [==============================] - 10s 52ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 3/5
186/186 [==============================] - 10s 52ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 4/5
186/186 [==============================] - 10s 53ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 5/5
186/186 [==============================] - 10s 53ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028


변화 없음

### 의문
> 혹시 epochs가 너무 적어서 학습이 안되는지 확인

In [391]:
hist = model.fit(train, test, epochs=20, batch_size=128, validation_data=(sx_val, sy_val))

Epoch 1/20
186/186 [==============================] - 10s 56ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 2/20
186/186 [==============================] - 11s 58ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 3/20
186/186 [==============================] - 10s 53ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 4/20
186/186 [==============================] - 9s 50ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 5/20
186/186 [==============================] - 9s 51ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 6/20
186/186 [==============================] - 9s 50ms/step - loss: 0.0000e+00 - accuracy: 0.4038 - val_loss: 0.0000e+00 - val_accuracy: 0.4028
Epoch 7/20
186/186 [==============================] - 10s 51ms/step - loss: 0.0000e+00 - ac

변화 없음